In this notebook we will do a quick analysis of the surface of a sundial. We will:

- Download Sundial pointcloud data
- Load it
- Visualize the points

Then we will look at how to fit a plane to part of the points:

- Filter out a plane section using a "thick plane"
- Visualize filtered points
- Fit a Plane shape to that filtered out section
- Plot the shape together with the filtered points

Finally, we'll have a look at a similar process of fitting a cone:

- Filter out a cone section using simple linear filtering
- Fit a Cone shape to the filtered points
- Plot the fitted shape
- Show the residuals of the fit
- Use a "thick cone" to refine the filtering of points
- Do another fitting iteration

In [ ]:
import numpy as np
import ipyvolume as ipv
import ectopylasm as ep

# Download data

The Topoi repository has a wealth of sundial scans freely available for download (under CC BY-NC-SA 3.0 DE license). We will download the following example, either with curl (only on Unix systems) or with Python urllib:

In [ ]:
# !curl -fLo ObjID126.ply http://repository.edition-topoi.org/BSDP/ReposBSDP/BSDP0030/ObjID126.ply

In [ ]:
# import urllib.request
# import shutil

# url = "http://repository.edition-topoi.org/BSDP/ReposBSDP/BSDP0030/ObjID126.ply"
# filename = "ObjID126.ply"
# with urllib.request.urlopen(url) as response, open(filename, 'wb') as out_file:
#     shutil.copyfileobj(response, out_file)

# Load data

The first time we load data from PLY files, `ectopylasm` will store an optimized version of the points (vertices) from the PLY file in a new HDF5 file with a `.cache.ecto` extension. The next time the PLY file is loaded, this will increase loading time significantly. This is all done under the hood, the user doesn't have to deal with this.

In [ ]:
points = ep.pandas_vertices_from_plyfile('ObjID126.ply')

# Visualize the points

Let's see what we've got!

In this notebook we use `ipyvolume` for plotting. All the `ectopylasm` shape plotting functions work with `ipyvolume` as well. For plotting pointclouds, one could also use `pptk`, which has a higher framerate, but is not integrated into the notebook, and doesn't support plotting shape surfaces.

In [ ]:
ep.pptk_plot_df(points)

In [ ]:
ipv.clear()
ep.ipv_plot_df(points)
ipv.show()

# Filter out a plane section

The bottom front part of the sundial seems like it's planar. Let's try to isolate that part and fit it to an actual plane.

In [ ]:
# estimate the parameters of the plane that encompasses our region
plane_point = (0, -70, -200)
plane_normal = (0, -1, 1)

In [ ]:
plane = ep.Plane.from_point(*plane_normal, plane_point)

In [ ]:
ipv.clear()
ep.ipv_plot_df(points)
ep.plot_plane(plane)
ipv.show()

That's not really it yet, let's adjust a bit.

In [ ]:
# tweak the parameters of the plane until the result looks good enough for filtering
plane_point = (0, -70, -200)
plane_normal = (0, -1, 0.7)

plane = ep.Plane.from_point(*plane_normal, plane_point)

ipv.clear()
ep.ipv_plot_df(points)
ep.plot_plane(plane)
ipv.show()

Looks good enough for now. Let's turn that into a filter then, shall we? We only need to estimate still the thickness. Something like 20-50 seems reasonable.

In [ ]:
filtered_points = np.array(ep.filter_points_plane(points.values.T, plane, 40)).T

In [ ]:
len(points), filtered_points.shape

In [ ]:
ipv.clear()
ep.ipv_plot_df(points)
ipv.scatter(*filtered_points, marker='circle_2d', size=0.4, color='blue')
ipv.show()

Ok, we took in a little bit too much. Let's manually filter out the junk we don't want to fit to with some simple conditionals.

In [ ]:
condition = np.logical_and(filtered_points[0] < 50, filtered_points[0] > -70)
condition = np.logical_and(condition, filtered_points[2] < -140)
condition = np.logical_and(condition, filtered_points[2] > -220)
filtered_points_2 = filtered_points.T[condition].T

In [ ]:
ipv.clear()
ep.ipv_plot_df(points)
ipv.scatter(*filtered_points_2, marker='circle_2d', size=0.4, color='blue')
ipv.show()

That's a nice planar sample.

# Fit a plane

Let's fit a plane to this section to find its parameters.

In [ ]:
fit_result = ep.fit_plane(filtered_points_2)

# Visualize results

Finally, let's see what we've got!

First we print the parameters, then we inspect the fit compared to the filtered points visually.

In [ ]:
print(fit_result)

In [ ]:
ipv.clear()
ipv.scatter(*filtered_points_2, marker='circle_2d', size=0.4, color='blue')
ep.plot_plane_fit(fit_result)
ipv.show()

As we can see, the fit is really good. We can use the plane parameters to do further analysis.

# Same for cone

The top part of the structure actually looks like some kind of conal section. Could we fit a cone to this part? Let's try!

For the filtering, we're just going to start with a rough coordinate slice, because guessing the cone parameters will be hard. The apex will be somewhere outside of the space.

In [ ]:
condition = np.logical_and(points.y < -10, points.z > -100)
condition = np.logical_and(condition, points.z < -20)
condition = np.logical_and(condition, points.x < 65)
condition = np.logical_and(condition, points.x > -85)
cone_filtered_points = points[condition]

In [ ]:
ipv.clear()
ep.ipv_plot_df(cone_filtered_points)
ipv.show()

Now, fitting this naively will take a very long time. It makes sense to provide some initial guesses to help the fitter along.

In [ ]:
# don't just run naively!
# fit_cone_result = ep.fit_cone(cone_filtered_points)

In [ ]:
# run with a good initial guess:
guess_cone = ep.Cone(300, 300, rot_x=-np.pi/3, base_pos=ep.Point(0, -260, -120))

In [ ]:
ipv.clear()
ep.ipv_plot_df(cone_filtered_points)
ep.plot_cone(guess_cone)
ipv.show()

Also, for performance, let's use just a random subset of all points.

In [ ]:
cone_points_sample = cone_filtered_points.sample(n=100)

In [ ]:
cone_points_sample.values.shape

In [ ]:
%time cone_fit = ep.Cone.from_points(cone_points_sample.values.T, initial_guess_cone=guess_cone)

In [ ]:
ipv.clear()
ep.ipv_plot_df(cone_filtered_points)
ep.ipv_plot_df(cone_points_sample, size=0.4, color='blue')
ep.plot_cone(cone_fit)
ipv.show()

Amazing, an absolutely perfect fit... These ancient Greeks were proper craftsmen.

Just for fun, let's see how that looks together with the full pointcloud.

In [ ]:
ipv.clear()
ep.ipv_plot_df(points)
ep.plot_cone(cone_fit)
ipv.show()

# Cone fit residuals

We can visualize the quality of the fit by calculating the distance of the points to the fitted cone and coloring our points to represent these distances. For instance, in the following way:

In [ ]:
distances = np.array([ep.point_distance_to_cone(point, cone_fit)[0] for point in cone_filtered_points.values])

In [ ]:
# map the distances to a 0, 1 scale
norm_distances = (distances - distances.min()) / (distances.max() - distances.min())

In [ ]:
import colorsys

In [ ]:
colors = [colorsys.hsv_to_rgb(distance, 1, 1) for distance in norm_distances]

In [ ]:
ipv.clear()
ep.ipv_plot_df(cone_filtered_points, size=0.4, color=colors)
ipv.show()

We can see that the fit is not perfect, which may either indicate the quality of the fit or could be due to actual defects in the sundial.

# Use thick cone filter

Using the distances we just calculated, we can get an idea of the thickness of a thick cone we should aim for to get a good sample of all the points in the cone. This should give us an even better sample of all the points that make up the conal structure in the sundial.

In [ ]:
distances.min(), distances.max()

Given those numbers, we could aim for a thickness of 10, so that we will get 5 units depth on both sides. Let's try:

In [ ]:
%time all_distances = np.array([ep.point_distance_to_cone(point, cone_fit)[0] for point in points.values])

In [ ]:
# cone_filtered_points_2 = np.array([point for point in points.values
#                                    if ep.point_distance_to_cone(point, cone_fit)[0] < 5])
cone_filtered_points_2 = points[all_distances < 5]

In [ ]:
len(cone_filtered_points_2)

In [ ]:
ipv.clear()
ep.ipv_plot_df(cone_filtered_points_2)
ep.plot_cone(cone_fit)
ipv.show()

There's two problems left here:

1. There is some crap included at the bottom, due to the fact that our cone is too elongated. We should adjust the cone's height to get a better sample.
2. The thickness of the filter causes us to include the parts around the edges of the cone. How to filter these out?

Let's start by defining a better cone. Mainly we want to shorten the height, by about a factor 0.4, I'd guess. We can then recalculate the radius from the current fit cone using its opening angle and by displacing it along the axis direction so that the apex stays in place:

In [ ]:
height_factor = 0.42

opening_angle = cone_fit.opening_angle()
desired_height = height_factor * cone_fit.height
desired_radius = desired_height * np.tan(opening_angle)

displacement = cone_fit.axis() * (1 - height_factor) * cone_fit.height
base_pos = ep.Point(cone_fit.base_pos.x + displacement[0],
                    cone_fit.base_pos.y + displacement[1],
                    cone_fit.base_pos.z + displacement[2])

filtering_cone = ep.Cone(desired_height, desired_radius,
                         rot_x=cone_fit.rot_x, rot_y=cone_fit.rot_y,
                         base_pos=base_pos)

In [ ]:
ipv.clear()
ep.ipv_plot_df(points)
ep.plot_cone(filtering_cone)
ipv.show()

Neat. Now to use that for filtering.

In [ ]:
%time all_distances_2 = np.array([ep.point_distance_to_cone(point, filtering_cone)[0] for point in points.values])

In [ ]:
cone_filtered_points_3 = points[np.array(all_distances_2) < 5]

In [ ]:
cone_filtered_points_3.shape

In [ ]:
ipv.clear()
ep.ipv_plot_df(cone_filtered_points_3)
ep.plot_cone(filtering_cone)
ipv.show()

Now we're getting somewhere. We still have the edge problems, but maybe it's not such a big problem, since most of the points are on the cone.

# Fitting: second try

Ok, second iteration. This time, we use our previously fitted cone as the initial guess. For performance reasons, we take only a sample of 200 points out of our filtered sample of ~80000.

In [ ]:
cone_points_sample_2 = cone_filtered_points_3.sample(n=200)

In [ ]:
ipv.clear()
ep.ipv_plot_df(cone_points_sample_2, size=1)
ep.plot_cone(filtering_cone)
ipv.show()

In [ ]:
%time cone_fit_2 = ep.Cone.from_points(cone_points_sample_2.values.T, initial_guess_cone=cone_fit)

In [ ]:
ipv.clear()
ep.ipv_plot_df(cone_filtered_points_3)
ep.plot_cone(cone_fit_2)
ipv.show()

Interesting, it does not seem as fixated on the outstanding edges as I had feared. What about the residuals?

In [ ]:
distances_2 = np.array([ep.point_distance_to_cone(point, cone_fit)[0] for point in cone_filtered_points_3.values])
# map the distances to a 0, 1 scale
norm_distances_2 = (distances_2 - distances_2.min()) / (distances_2.max() - distances_2.min())
colors_2 = [colorsys.hsv_to_rgb(distance, 1, 1) for distance in norm_distances_2]

In [ ]:
ipv.clear()
ep.ipv_plot_df(cone_filtered_points_3, size=0.4, color=colors_2)
ipv.show()

Hm, weird, why is that thingy still in there? Wait, did I use absolute distances? Probably not ;)

# Again!

In [ ]:
cone_filtered_points_4 = points[np.abs(all_distances_2) < 5]

In [ ]:
cone_filtered_points_4.shape

In [ ]:
# let's skip "_3" now to make life a bit less confusing...
cone_points_sample_4 = cone_filtered_points_4.sample(n=200)

In [ ]:
ipv.clear()
ep.ipv_plot_df(cone_points_sample_4, size=1)
ep.plot_cone(filtering_cone)
ipv.show()

In [ ]:
%time cone_fit_4 = ep.Cone.from_points(cone_points_sample_4.values.T, initial_guess_cone=cone_fit)

Already, a wall clock time 3 times shorter tells us the fit was probably easier.

In [ ]:
ipv.clear()
ep.ipv_plot_df(cone_filtered_points_4)
ep.plot_cone(cone_fit_4)
ipv.show()

In [ ]:
distances_4 = np.array([ep.point_distance_to_cone(point, cone_fit)[0] for point in cone_filtered_points_4.values])
# map the distances to a 0, 1 scale
norm_distances_4 = (distances_4 - distances_4.min()) / (distances_4.max() - distances_4.min())
colors_4 = [colorsys.hsv_to_rgb(distance, 1, 1) for distance in norm_distances_4]

In [ ]:
ipv.clear()
ep.ipv_plot_df(cone_filtered_points_4, size=0.4, color=colors_4)
ipv.show()

Really interesting how the lines are so well visible.